### Parse `Common-Rig`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.6]


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Parse `common_rig.xml`

In [2]:
xml_path = '../asset/myo_sim/human/myohuman_v0.1(mj236).mjb'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=True)

dt:[0.0020] HZ:[500]
n_dof (=nv):[83]
n_geom:[82]
geom_names:['floor', 'r_pelvis', 'l_pelvis', 'sacrum', 'r_femur', 'r_tibia', 'r_fibula', 'r_talus', 'r_foot', 'r_bofoot', 'r_patella', 'l_femur', 'l_tibia', 'l_fibula', 'l_talus', 'l_foot', 'l_bofoot', 'l_patella', 'hat_spine', 'hat_jaw', 'hat_skull', 'hat_ribs_scap', 'humerus_r', 'ulna', 'radius', 'lunate', 'scaphoid', 'pisiform', 'triquetrum', 'capitate', 'trapezium', '1mc', 'thumbprox', 'thumbdist', 'trapezoid', 'hamate', '2mc', '2proxph', '2midph', '2distph', '3mc', '3proxph', '3midph', '3distph', '4mc', '4proxph', '4midph', '4distph', '5mc', '5proxph', '5midph', '5distph', 'humerus_lv', 'ulna_l', 'radius_l', 'lunate_l', 'scaphoid_l_geom_1', 'pisiform_l_geom_1', 'triquetrum_l_geom_1', 'capitate_l_geom_1', 'trapezium_l_geom_1', 'firstmc_l_geom_1', 'proximal_thumb_l_geom_1', 'distal_thumb_l_geom_1', 'trapezoid_l_geom_1', 'hamate_l_geom_1', 'secondmc_l_geom_1', '2proxph_l_geom_1', '2midph_l_geom_1', '2distph_l_geom_1', 'thirdmc_l_geom_

### Modify the `common-rig` model

In [3]:
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue 
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

Done.


### Loop

In [5]:
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()

while env.is_viewer_alive():
    
    # Simulate
    rev_joint_names = ['deviation_l','flexion_l','pro_sup']
    q_fwd = np.radians([0,0,0])
    joint_idxs_fwd = [env.model.joint(jname).qposadr[0] for jname in rev_joint_names]
    env.forward(q=q_fwd,joint_idxs=joint_idxs_fwd) # kinematic simulation
    
    if env.loop_every(HZ=10):
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        
        # Plot bodies
        for body_name in env.body_names:
            p,R = env.get_pR_body(body_name=body_name)
            env.plot_T(p=p,R=R,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
        
        # Plot revolute joints with arrow
        for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
            axis_joint = env.model.jnt_axis[rev_joint_idx]
            p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
            axis_world = R_joint@axis_joint
            axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
            axis_len,axis_r = 0.1,0.01
            env.plot_arrow_fr2to(
                p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                r=axis_r,rgba=axis_rgba)
        # Plot contact information
        env.plot_contact_info(h_arrow=0.3,rgba_arrow=[1,0,0,1],
                              PRINT_CONTACT_BODY=False)
        env.render()
        
# Close viewer
env.close_viewer()

In [5]:
# env.open_interactive_viewer()